## Numbers of K

In the previous lesson, we managed to write a single, complete function capable of splitting any DataFrame into as many clusters as we want to. This function, which we named kmeans, receives three arguments: df, k, and n_iterations:

- df is the two-column DataFrame we want to split
- k is the number of clusters
- n_iterations is the maximum number of times the function should run while trying to find the convergence point.

The function plots the clustered dataset in a scatter plot and returns a Series containing the cluster for each data point.

Having already written the function, we've had time to experiment with different combinations of inputs. The results of the segmentation change as we change the variables, especially the number of clusters, which brings us to a question: which number should we use for k?

Occasionally, we may find ourselves in a position where the number of clusters is defined by a business rule. For instance, we may be asked to split customers given a particular set of variables into exactly two groups because the commercial and marketing teams only have the budget for two different advertising strategies. In this case, we have k=2 already defined.

However, most of the time, the number of clusters will not be determined for us, and we'll have to find the best number of clusters for the dataset.

In this lesson, we'll learn the metrics and techniques used to evaluate whether the number of clusters is good enough and how to decide the best number for our dataset. We'll build on everything we've done so far to create the best possible split for the customers' data we are working with.

## Inertia

One of the most important metrics for evaluating a means algorithm is inertia. Inertia reflects how far the data points assigned to a cluster are from that cluster's centroid. Mathematically, inertia is the sum of the squared distances from each of `n` data points to a centroid.

Notice that the formula uses the squared distance not only to keep all the numbers positive, but also to make sure that the more distant a point is to the cluster, the more the cluster will be penalized. This means that a cluster with some points very close to the centroid and some points very far from the centroid will present higher inertia than a cluster with points located at a medium distance from the centroid. Check the example below:

![](https://s3.amazonaws.com/dq-content/743/2.1-m743.svg)

In the example, both clusters have four observations assigned to them. However, the blue cluster has better-distributed points. If we do the math, the distance from each point to the blue centroid is 5, which produces inertia equal to 100. On the other side, the red cluster has two points very close and two points very far from the centroid. The distances from each point to the red centroid are 1, 1, 9, and 12.04, which produces inertia equal to 228.

Inertia tells us that the blue cluster is much better than the red one because it's better distributed.

Inertia and the number of clusters are inversely proportional, which means that the lower the number of clusters, the higher the inertia.

For instance, consider a dataset with one hundred observations and imagine that we split it into one hundred clusters. This means that each observation is its own cluster. Therefore, the distance from each data point to its cluster centroid is zero, and the inertia is also zero. But if each one is a centroid, then none of them is a centroid, and we have no split at all.

On the other hand, if we split these one hundred observations into 10 clusters, the inertia will be greater than the zero we had when using one hundred clusters. However, it makes more sense to split a one hundred-row dataset into ten clusters than into one hundred, even though the inertia is higher.

The goal is to find the sweet spot in the inertia-clusters trade-off. We need to find the lower number of clusters that produces the lowest inertia and still makes sense to split the data with.

We have already covered calculating the distance between two points; now we can focus on the calculation of inertia.

### Instructions
The kmeans function from the previous lesson is already defined, as well as the other functions we built.

1. Modify the kmeans function so that:

- After the iterations are over, it creates a new column in df called sqrt_dist_centroid. For each row, this column should have the squared distance from the row to the centroid.

- It calculates the sum of the sqrt_dist_centroid column and assigns it to inertia.

- It returns inertia alongside df['cluster'].

2. Call the modified kmeans function using the customers DataFrame and k=2. Assign the results to clusters and inertia and print them.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


customers = pd.read_csv('mall_customers.csv')

cols_to_keep = ['Annual Income', 'Spending Score']
customers = customers[cols_to_keep].copy()

def kmeans(df, k, n_iterations=100):
    variables = df.columns

    centroids, coords = get_centroids(df, k)

    for i in range(n_iterations):
        last_coords = coords.copy()

        df, dists = calculate_distance(df, coords)

        df['cluster'] = df[dists].idxmin(axis=1).str.split('_').str[-1]

        centroids = round(df.groupby('cluster')[variables].mean(), 4)
        coords = centroids.values.tolist()

        if last_coords == coords:
          	break
        
    df['sqrt_dist_centroid'] = df[dists].min(axis=1)**2
    inertia = df['sqrt_dist_centroid'].sum()
    
    print(f'Total Iterations: {i + 1}')

    fig, ax = plt.subplots(figsize=(10, 5))
    sns.scatterplot(variables[0], variables[1], hue='cluster', palette='tab10', data=df, s=50, ax=ax)
    sns.scatterplot(variables[0], variables[1], color='black', data=centroids, s=100, ax=ax)


    plt.tight_layout()
    plt.show()

    return df['cluster'], inertia


clusters, inertia = kmeans(customers, k=2)
print(clusters, inertia)

## Calculating More Inertias

In the previous screen, we performed the clustering, but this time we also calculated the inertia for the clusters we created.

The result of inertia was a big number--over 184 thousand. But over 184 thousand what? As we're not using any particular unit of measurement, since we're calculating the distance between units of annual income and units of spending score, we can say that the inertia was over 184 thousand units of distance.

But this number doesn't make much sense by itself. In order to better understand this number, we'll compare it to other inertias on the same data.

The 184 thousand inertia refers to the split of the customers dataset between two clusters. The next step is to calculate the inertia for other numbers of clusters, which will make us capable of comparing and deciding whether or not 184 thousand is a big number and what a reasonable number for inertia should be.

### Instructions
Initialize an empty list called inertias.

Inside a for loop that goes from one to ten, call the kmeans function using the customers DataFrame and the number of the clusters as the iterable variable.

Append the resulting inertia to the inertias list.

Print inertias.

Try to draw some conclusions from the inertias. Are they getting higher or lower?

The kmeans function was modified and now includes a boolean plot parameter that allows us to choose whether or not we want to plot the clusters so the code doesn't plot the charts in each iteration. This parameter is set to default False.

In [ ]:
inertias = []
for k in range(1,11):
    clusters, inertia = kmeans(customers, k=k)
    inertias.append(inertia)
    
print(inertias)

## The Elbow Curve
From the results of the previous screen, the inertia decreased as the number of clusters increased. That makes perfect sense with what we discussed before: if the number of clusters keeps increasing until it reaches the number of observations in the dataset, inertia will be zero.

But when is it enough? At which point should we say that the inertia has decreased to a point where we're comfortable using that number of clusters?

Keeping in mind that the ideal model has few clusters and low inertia, we can see the number of clusters as the cost of lowering inertia.

In order to increase our cost by increasing the number of clusters by one, we must have a gain in inertia or, in other words, a better model. We should stop adding clusters when the gain in inertia increases at lower and lower rates. In economics, this is called diminishing returns.

That's when the Elbow Curve comes in. The Elbow Curve is nothing more than a line plot of the inertias against the number of clusters. Since the values on the y-axis are decreasing while the values on the x-axis are increasing, the curve should look like this:

![](https://s3.amazonaws.com/dq-content/743/4.1-m743.svg)

For this particular use case, we call it the elbow curve because we assume that the first clusters will cause the inertia to decrease very fast, and, at some point, this decrease will become slower in such a way that we'll see a sharp "elbow" on the curve:

![](https://s3.amazonaws.com/dq-content/743/4.2-m743.svg)

This sharp "elbow" is where we should set our number of clusters.

The bottom line is that the Elbow Curve allows us to visually understand how the number of clusters affects the quality of the model. As we said in the previous screen, a single value for inertia does not say anything, but a visual representation of many inertias put together makes it a lot easier to set the number of clusters for our model.

### Instructions
The list of inertias calculated on the previous screen is already displayed.

Import matplotlib.pyplot as plt.

Use this list to plot the Elbow Curve.

The y-axis should be the values in the list, while the x-axis should be the number of clusters from one to ten.

Pass marker='o' to plt.plot to better visualize the points on the curve.

Can you find a sharp "elbow"? Do you notice the diminishing returns? What should the number of clusters be for this dataset?

In [ ]:
inertias = [269981.27999999997, 184131.88502826, 106348.37306241, 
            73679.78903966, 44448.45544817, 38718.38226857,
            34918.93964226, 30176.132287570003, 29068.11150443,
            21063.88614989]


plt.plot(range(1, 11), inertias, marker='o')
plt.show()

## Choosing K
In the previous screen, we plotted the Elbow Curve for k going from one to ten. The elbow isn't very sharp, but a broader elbow made of three points.

![](https://s3.amazonaws.com/dq-content/743/5.1-m743.svg)

That's actually the most common scenario we'll deal with when working with k-means and inertia. Occasionally, we'll see a very sharp elbow that leaves no doubt about the number of clusters, but that's not the most common scenario.

In this case, all three points would've been a reasonable acceptable choice. Notice that there are decreases from 1 to 2 and from 2 to 3. The decreases from 3 to 4 and 4 to 5 are smaller. It's from point 5 to point 6 that we have the biggest decrease in inertia.

Notice the line becomes almost horizontal at this point. If we do the math, that's actually the lowest percent loss of inertia. The image below shows the percent decrease in inertia, and we move to each new number of K:

![](https://s3.amazonaws.com/dq-content/743/5.2-m743.svg)

Notice that until k=5, the decrease is much higher, and it's going up and down, which means it's not decreasing at lower and lower rates. From the fifth to the sixth point, the decrease is dramatically lower.

Finally, clustering is usually related to business and operations areas in companies, since we're frequently looking to segment customers, stores, products, purchases, and more.

For instance, let's say we're segmenting these customers so each cluster can have its advertisement strategy held by a single team in the marketing department. However, the marketing department only has four of these teams available right now. With this information, we can discard the strategy of using five clusters.

In another example, let's say we're not segmenting customers, but stores in a big chain of supermarkets in our city. The company has built five distribution centers, strategically located around the city, and, although a distribution center can service up to two clusters, they all need to have at least one cluster assigned to them. This means we would need a minimum of five clusters and by looking at the curve, we would definitely choose five.

Always be aware of input from business teams when making decisions about the ideal number of clusters for your problem.

### Instructions
1. Use the kmeans function to segment the data into 5 clusters using the Annual Income and Spending Score variables from the customers DataFrame. Assign the outputs of the function to clusters and inertia.
2. Check the total number of iterations printed by the function. What conclusion can you draw from all this? Do the clusters look better split with five clusters than two? What was the increase in the number of iterations? Is the inertia actually lower than before?

In [ ]:
customers = pd.read_csv('mall_customers.csv')

cols_to_keep = ['Annual Income', 'Spending Score']
customers = customers[cols_to_keep].copy()

clusters, inertia = kmeans(customers, k=5, plot=True)

## Next Steps
In this lesson, we finished building a k-means algorithm from scratch and also became familiar with the concept of inertia and how to use the Elbow Curve.

In next the lesson, we'll get to know scikit-learn's implementation of k-means and how to analyze and interpret the results of a clusterization